Related YouTube Video:
https://youtu.be/JHzFrujpBz0

In [ ]:
import os
import whisper
from pytube import YouTube
import pandas as pd

In [ ]:
model = whisper.load_model('base')

In [ ]:
youtube_video_url = "https://www.youtube.com/watch?v=Q2u0dKRA1cA"
youtube_video = YouTube(youtube_video_url)

In [ ]:
dir(youtube_video)

In [ ]:
watch_url = youtube_video.watch_url
watch_url

In [ ]:
video_id = youtube_video.video_id
video_id

In [ ]:
title = youtube_video.title
title

In [ ]:
video_data = {
    'title': youtube_video.title,
    'views': youtube_video.views,
    'length': youtube_video.length,
    'publish_date': youtube_video.publish_date,
    'watch_url': youtube_video.watch_url,
}
video_df = pd.DataFrame([video_data])
video_df


In [ ]:
streams = youtube_video.streams
streams

In [ ]:
for stream in streams:
    print(stream)

In [ ]:
audio_streams = youtube_video.streams.filter(only_audio=True)
audio_streams

In [ ]:
for audio_stream in audio_streams:
    print(audio_stream)

In [ ]:
stream = audio_streams.first()
stream

In [ ]:
stream.download(filename=f"./mp4/{video_id}.mp4")

In [ ]:
# do the transcription
output = model.transcribe(f"./mp4/{video_id}.mp4", fp16=False)

In [ ]:
output

In [ ]:
output_text = output["text"]
output_text

In [ ]:
# Path to Obsidian vault
obsidian_dir = r'C:/1600/2nd_brain/1600_agent_swarm/auto24/transcription/'

# Combine the prefix and formatted date to create the filename
filename = f"{title}.md"
    
# full path prefix obsidian_dir to filename
full_path = os.path.join(obsidian_dir, filename)

# Write content to file
with open(full_path, 'w') as file:
    file.write(output_text)

In [ ]:
segments = output["segments"]
segments

In [ ]:
type(segments)

In [ ]:
df = pd.DataFrame(segments)
df.head(5)

In [ ]:
# Cache the transcription
df.to_csv(f"./csv/{video_id}.csv", index=False)

In [ ]:
df_backup = pd.read_csv(f"./csv/{video_id}.csv")
df_backup.head(3)

In [ ]:
df_export = df_backup[['id', 'start', 'text']]
df_export.head(5)

In [ ]:
df_export.insert(0, 'video_id', video_id)

In [ ]:
df_export.head(5)

In [ ]:
df_export.insert(0, 'watch_url', watch_url)

In [ ]:
df_export.head(5)

In [ ]:
# Lambda function to modify the URL column
add_time = lambda row: row['watch_url'] + '&t=' + str(int(row['start']))

In [ ]:
df_export.loc[:, 'watch_url'] = df_export.apply(add_time, axis=1)

In [ ]:
df_export

In [ ]:
# Path to Obsidian vault
obsidian_dir = r'C:/1600/2nd_brain/1600_agent_swarm/auto24/transcription/'

# Combine the prefix and formatted date to create the filename
filename = f"{title}_table.md"
    
# full path prefix obsidian_dir to filename
full_path = os.path.join(obsidian_dir, filename)

# Write content to file
df_export.to_markdown(full_path)

In [ ]:
# Function to format a column as a hyperlink
def format_as_hyperlink(value):
    return f'=HYPERLINK("{value}", "{value}")'

In [ ]:

# Apply hyperlink formatting to the 'URL' column
df_export.loc[:, 'watch_url'] = df_export['watch_url'].apply(format_as_hyperlink)



In [ ]:
df_export

In [ ]:
df_export.to_excel(f"./xlsx/{video_id}.xlsx", sheet_name=video_id, index=False)